CARGAS

In [21]:
import sqlite3

# Conectar/criar banco de dados SQLite local
conn = sqlite3.connect('cotation.db')
cursor = conn.cursor()

# Criar tabela BASE_COTATION
cursor.execute('''
CREATE TABLE IF NOT EXISTS BASE_COTATION (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    category TEXT,
    product TEXT,
    description TEXT,
    ncm TEXT,
    value REAL,
    comments TEXT
)
''')

# Dados fornecidos (convertendo valor para float, tirando "R$" e espaços)
dados = [
    (" ", " ", " ", " ", 0, None),
    ("Adesivos e Selantes", "Adesivo instantâneo (cianoacrilato)", "Adesivo instantâneo base cianoacrilato, cura rápida, alta resistência, uso geral. Embalagem: 20g ou 22g. Ex: Super Bonder Loctite.", "3506.10.10", 300.00, None),
    ("Adesivos e Selantes", "Adesivo de contato (Cascola 730 g)", "Adesivo de contato à base de borracha sintética, marca Cascola, para colagem de MDF, couro, EVA, etc. Embalagem: lata 730g.", "3506.10.90", 238.00, None),
    ("Adesivos e Selantes", "Adesivo Cascola (30 g)", "Adesivo de contato Cascola tradicional, base solvente, para pequenos reparos. Embalagem: tubo 30g.", "3506.91.10", 187.00, None),
    ("Adesivos e Selantes", "Adesivo Cascola (70 g)", "Adesivo de contato Cascola tradicional, base solvente, para pequenos reparos. Embalagem: tubo 70g.", "3402.90.15", 250.00, None),

]

# Inserir os dados na tabela
cursor.executemany('''
INSERT INTO BASE_COTATION (category, product, description, ncm, value, comments)
VALUES (?, ?, ?, ?, ?, ?)
''', dados)

# Salvar (commit) as alterações e fechar conexão
conn.commit()
conn.close()

print("Banco e tabela BASE_COTATION criados com sucesso e dados inseridos.")


Banco e tabela BASE_COTATION criados com sucesso e dados inseridos.


In [22]:
import sqlite3

# Conectar ao banco
conn = sqlite3.connect("cotation.db")
cursor = conn.cursor()

# Executar consulta
cursor.execute("SELECT * FROM BASE_COTATION")

# Buscar todos os resultados
registros = cursor.fetchall()

# Mostrar registros
for registro in registros:
    print(registro)

# Fechar conexão
conn.close()


(1, ' ', ' ', ' ', ' ', 0.0, None)
(2, 'Adesivos e Selantes', 'Adesivo instantâneo (cianoacrilato)', 'Adesivo instantâneo base cianoacrilato, cura rápida, alta resistência, uso geral. Embalagem: 20g ou 22g. Ex: Super Bonder Loctite.', '3506.10.10', 300.0, None)
(3, 'Adesivos e Selantes', 'Adesivo de contato (Cascola 730\u202fg)', 'Adesivo de contato à base de borracha sintética, marca Cascola, para colagem de MDF, couro, EVA, etc. Embalagem: lata 730g.', '3506.10.90', 238.0, None)
(4, 'Adesivos e Selantes', 'Adesivo Cascola (30\u202fg)', 'Adesivo de contato Cascola tradicional, base solvente, para pequenos reparos. Embalagem: tubo 30g.', '3506.91.10', 187.0, None)
(5, 'Adesivos e Selantes', 'Adesivo Cascola (70\u202fg)', 'Adesivo de contato Cascola tradicional, base solvente, para pequenos reparos. Embalagem: tubo 70g.', '3402.90.15', 250.0, None)


In [1]:
import sqlite3
import pandas as pd
import os

# Nome dos arquivos
DB_FILE = "cotation.db"
EXCEL_FILE = "DB_PRODUCTS.xlsx"

print(f"📂 Banco que será usado: {os.path.abspath(DB_FILE)}")
print(f"📂 Planilha que será usada: {os.path.abspath(EXCEL_FILE)}")

# Ler planilha
df = pd.read_excel(EXCEL_FILE)

# Conferir colunas esperadas
colunas_esperadas = ["category", "product", "description", "ncm", "value", "comments"]
if not all(col in df.columns for col in colunas_esperadas):
    raise ValueError(f"A planilha deve conter as colunas: {colunas_esperadas}")

# Limpar espaços extras e garantir string
df["product"] = df["product"].astype(str).str.strip()

# Conectar ao banco
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Atualizar ou inserir
for _, row in df.iterrows():
    produto = row["product"].strip()
    cursor.execute("""
        SELECT id FROM BASE_COTATION WHERE TRIM(LOWER(product)) = TRIM(LOWER(?))
    """, (produto,))
    resultado = cursor.fetchone()

    if resultado:  
        print(f"🔄 Atualizando: {produto}")
        cursor.execute("""
            UPDATE BASE_COTATION
            SET category = ?, description = ?, ncm = ?, value = ?, comments = ?
            WHERE TRIM(LOWER(product)) = TRIM(LOWER(?))
        """, (
            row["category"], row["description"], row["ncm"], row["value"], row["comments"], produto
        ))
    else:
        print(f"➕ Inserindo novo: {produto}")
        cursor.execute("""
            INSERT INTO BASE_COTATION (category, product, description, ncm, value, comments)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (
            row["category"], produto, row["description"], row["ncm"], row["value"], row["comments"]
        ))

# Salvar alterações e fechar
conn.commit()
conn.close()

print("✅ Tabela BASE_COTATION atualizada com sucesso a partir da planilha.")


📂 Banco que será usado: y:\AMY\PROG\DS_FORMULARIO\cotation.db
📂 Planilha que será usada: y:\AMY\PROG\DS_FORMULARIO\DB_PRODUCTS.xlsx
🔄 Atualizando: Graxa Pasta tubo 100g ou pote 1kg, Molykote
🔄 Atualizando: Graxa Pasta tubo 3,5kg, Molykote
🔄 Atualizando: Adesivo instantâneo (cianoacrilato)
🔄 Atualizando: Adesivo de contato (Cascola 730 g)
🔄 Atualizando: Adesivo Cascola (30 g)
🔄 Atualizando: Abraçadeira de nylon
🔄 Atualizando: Abraçadeira metálica
🔄 Atualizando: Saco alvejado
🔄 Atualizando: Arame farpado
🔄 Atualizando: Ventilador de parede
🔄 Atualizando: Ventilador pedestal
🔄 Atualizando: Geladeira Frost Free
🔄 Atualizando: Chave parafusadeira
🔄 Atualizando: qwe
✅ Tabela BASE_COTATION atualizada com sucesso a partir da planilha.


In [2]:
import sqlite3

# Conectar ao banco
conn = sqlite3.connect("cotation.db")
cursor = conn.cursor()

# Executar consulta
cursor.execute("SELECT * FROM BASE_COTATION")

# Buscar todos os resultados
registros = cursor.fetchall()

# Mostrar registros
for registro in registros:
    print(registro)

# Fechar conexão
conn.close()


(1, ' ', ' ', ' ', ' ', 0.0, None)
(2, 'Adesivos e Selantes', 'Adesivo instantâneo (cianoacrilato)', 'Adesivo instantâneo base cianoacrilato, cura rápida, alta resistência, uso geral. Embalagem: 20g ou 22g. Ex: Super Bonder Loctite.', '3506.10.10', 125.0, None)
(3, 'Adesivos e Selantes', 'Adesivo de contato (Cascola 730\u202fg)', 'Adesivo de contato à base de borracha sintética, marca Cascola, para colagem de MDF, couro, EVA, etc. Embalagem: lata 730g.', '3506.10.90', 200.0, None)
(4, 'Adesivos e Selantes', 'Adesivo Cascola (30\u202fg)', 'Adesivo de contato Cascola tradicional, base solvente, para pequenos reparos. Embalagem: tubo 30g.', '3506.91.10', 90.0, None)
(5, 'Adesivos e Selantes', 'Adesivo Cascola (70\u202fg)', 'Adesivo de contato Cascola tradicional, base solvente, para pequenos reparos. Embalagem: tubo 70g.', '3402.90.15', 250.0, None)
(6, 'Lubrificantes Industriais', 'Graxa Pasta tubo 100g ou pote 1kg, Molykote', 'Graxa lubrificante tipo pasta, marca Molykote, base sintétic

In [23]:
import sqlite3

# Conectar ao banco cotation.db
conn = sqlite3.connect('cotation.db')
cursor = conn.cursor()

# Criar tabela email_db
cursor.execute('''
CREATE TABLE IF NOT EXISTS email_db (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    comprador TEXT NOT NULL,
    email TEXT
)
''')

# Dados dos compradores e emails
dados_email = [
    (" ", " "),
    ("ELIANE NOLASCO", "elianee@eletronuclear.gov.br"),
    ("GISELE LACERDA", "giselel@eletronuclear.gov.br"),
    ("MERELLIN BULHÕES", "merelli@eletronuclear.gov.br"),
    ("POLIANA CASSIANO", "poli01@eletronuclear.gov.br"),
    ("MANOEL JORDÃO", "mjcar@eletronuclear.gov.br"),
    ("MARIA CLARA", "claraf@eletronuclear.gov.br"),
    ("GIOVANA OLIVEIRA", "gioliv@eletronuclear.gov.br"),
    ("LUIZ FELIPE", "lmendes@eletronuclear.gov.br"),
    ("MANUELA SANTOS", "manusan@eletronuclear.gov.br"),
    ("JOÃO PEDRO", "jpcunha@eletronuclear.gov.br"),
    ("JAQUELINE VASCONCELOS", "jaquasc@eletronuclear.gov.br"),
    ("HENRIQUE PASSOS", "henrique.passos@marteengenharia.com.br"),
    ("MARCOS VINÍCIOS", "vinicio@eletronuclear.gov.br"),
    ("FLÁVIA FLUTUOSO", "flacris@eletronuclear.gov.br"),
    ("ALUISIO PEYNEAU", "alupey@eletronuclear.gov.br")
]

# Inserir dados na tabela email_db
cursor.executemany('''
INSERT INTO email_db (comprador, email) VALUES (?, ?)
''', dados_email)

conn.commit()
conn.close()

print("Tabela email_db criada e dados inseridos com sucesso.")


Tabela email_db criada e dados inseridos com sucesso.


In [24]:
import sqlite3

# Conectar ao banco cotation.db
conn = sqlite3.connect('cotation.db')
cursor = conn.cursor()

# Executar consulta para selecionar todos os compradores e emails
cursor.execute('SELECT comprador, email FROM email_db')

# Buscar todos os resultados
rows = cursor.fetchall()

# Exibir os resultados
for comprador, email in rows:
    print(f"Comprador: {comprador} - Email: {email}")

# Fechar conexão
conn.close()


Comprador:   - Email:  
Comprador: ELIANE NOLASCO - Email: elianee@eletronuclear.gov.br
Comprador: GISELE LACERDA - Email: giselel@eletronuclear.gov.br
Comprador: MERELLIN BULHÕES - Email: merelli@eletronuclear.gov.br
Comprador: POLIANA CASSIANO - Email: poli01@eletronuclear.gov.br
Comprador: MANOEL JORDÃO - Email: mjcar@eletronuclear.gov.br
Comprador: MARIA CLARA - Email: claraf@eletronuclear.gov.br
Comprador: GIOVANA OLIVEIRA - Email: gioliv@eletronuclear.gov.br
Comprador: LUIZ FELIPE - Email: lmendes@eletronuclear.gov.br
Comprador: MANUELA SANTOS - Email: manusan@eletronuclear.gov.br
Comprador: JOÃO PEDRO - Email: jpcunha@eletronuclear.gov.br
Comprador: JAQUELINE VASCONCELOS - Email: jaquasc@eletronuclear.gov.br
Comprador: HENRIQUE PASSOS - Email: henrique.passos@marteengenharia.com.br
Comprador: MARCOS VINÍCIOS - Email: vinicio@eletronuclear.gov.br
Comprador: FLÁVIA FLUTUOSO - Email: flacris@eletronuclear.gov.br
Comprador: ALUISIO PEYNEAU - Email: alupey@eletronuclear.gov.br


In [25]:
import sqlite3

# Conectar ao banco cotation.db
conn = sqlite3.connect('cotation.db')
cursor = conn.cursor()

# Criar tabela UN_BASE
cursor.execute('''
CREATE TABLE IF NOT EXISTS UN_BASE (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    unit TEXT UNIQUE NOT NULL
)
''')

# Lista de unidades
units = [
    " ",
    "PEÇA",
    "UNIT",
    "BOBINA",
    "CJ",
    "FARDO",
    "FOLHA",
    "KG",
    "LITRO",
    "M²",
    "M³",
    "METRO",
    "METROS",
    "PACOTE",
    "PARES",
    "PEÇAS"
]

# Inserir unidades na tabela, ignorando duplicatas (devido UNIQUE)
for u in units:
    try:
        cursor.execute('INSERT INTO UN_BASE (unit) VALUES (?)', (u,))
    except sqlite3.IntegrityError:
        pass  # unidade já existe, ignora

# Salvar e fechar
conn.commit()
conn.close()

print("Tabela UN_BASE criada e unidades inseridas com sucesso.")

Tabela UN_BASE criada e unidades inseridas com sucesso.


In [26]:
import sqlite3

# Conectar ao banco
conn = sqlite3.connect('cotation.db')
cursor = conn.cursor()

# Executar consulta para buscar todas as unidades
cursor.execute('SELECT id, unit FROM UN_BASE')

# Buscar todos os resultados
rows = cursor.fetchall()

# Iterar e imprimir
for row in rows:
    print(f"ID: {row[0]} - Unidade: {row[1]}")

# Fechar conexão
conn.close()


ID: 1 - Unidade:  
ID: 2 - Unidade: PEÇA
ID: 3 - Unidade: UNIT
ID: 4 - Unidade: BOBINA
ID: 5 - Unidade: CJ
ID: 6 - Unidade: FARDO
ID: 7 - Unidade: FOLHA
ID: 8 - Unidade: KG
ID: 9 - Unidade: LITRO
ID: 10 - Unidade: M²
ID: 11 - Unidade: M³
ID: 12 - Unidade: METRO
ID: 13 - Unidade: METROS
ID: 14 - Unidade: PACOTE
ID: 15 - Unidade: PARES
ID: 16 - Unidade: PEÇAS
